In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DIPNA\\Desktop\\Kidny\\Kidney_Disease_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DIPNA\\Desktop\\Kidny\\Kidney_Disease_Classification'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = (
    "https://dagshub.com/Dippy2003/Kidney_Disease_Classification.mlflow"
)
# DAGsHub credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "Dippy2003"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "d2e2ba105fc8e6cf1eb0d249e886031d199019c5"


In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Dippy2003/Kidney_Disease_Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

c:\Users\DIPNA\anaconda3\envs\kidney\lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        """Log evaluation results + model to DagsHub MLflow."""
        # Ensure we use a named experiment on the remote server
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        mlflow.set_experiment("Kidney_Disease_Classification")

        # Normalize params (MLflow params must be str/number/bool)
        raw_params = self.config.all_params
        try:
            params_dict = dict(raw_params)
        except Exception:
            params_dict = {}

        normalized_params = {}
        for k, v in params_dict.items():
            if isinstance(v, (int, float, str, bool)):
                normalized_params[k] = v
            else:
                normalized_params[k] = str(v)

        with mlflow.start_run(run_name="evaluation"):
            mlflow.log_params(normalized_params)
            mlflow.log_metrics({"loss": float(self.score[0]), "accuracy": float(self.score[1])})
            mlflow.keras.log_model(self.model, "model")


In [13]:
from mlflow.exceptions import RestException

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except RestException as e:
    # Print the server-provided details (this is the real reason for INVALID_PARAMETER_VALUE)
    print("MLflow error_code:", getattr(e, "error_code", None))
    print("MLflow server json:", getattr(e, "json", None))
    raise
except Exception as e:
    raise e

[2026-02-04 23:22:23,506: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-04 23:22:23,510: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-04 23:22:23,512: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 9s 970ms/step - loss: 23.4747 - accuracy: 0.5468
[2026-02-04 23:22:33,330: INFO: common: json file saved at: scores.json]


2026/02/04 23:22:35 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-02-04 23:22:36,534: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\DIPNA\AppData\Local\Temp\tmpjouk0yuf\model\data\model\assets
[2026-02-04 23:22:36,920: INFO: builder_impl: Assets written to: C:\Users\DIPNA\AppData\Local\Temp\tmpjouk0yuf\model\data\model\assets]


c:\Users\DIPNA\anaconda3\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
